In [1]:
import re
import cobra
import pandas as pd
import itertools
import cobra.flux_analysis
import multiprocessing

"""A mess of a document with different code cells.
Good to to use for any testing that involves the Recon3D model, as it takes some time to load in."""

model_file = 'C:/Users/Sigve/Genome_Data/Recon3D/JSON/Recon3D.json'
model = cobra.io.load_json_model(model_file)

SNP_results = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNPs_effect_mod.tsv', index_col=0)

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic


In [2]:
# Prepare SNP combinations, reaction constraints
combinations = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNP_combinations.tsv', index_col=0)
combinations['combinations'] = combinations['combinations'].apply(lambda x: x.split(';'))

combinations['gene_model_ids'] = combinations['combinations']\
    .apply(lambda x: SNP_results.loc[SNP_results['variant_name'].isin(x), ['gene_number']].iloc[:, 0].tolist())

id_list = ' '.join(model.genes.list_attr('id'))

combinations['gene_model_ids'] = combinations['gene_model_ids'].apply(lambda x:
                                list(set(itertools.chain.from_iterable(
                                [re.findall(r"(?:\s)(" + str(i) + r"\w\w\w\d)", id_list) for i in x]))))

combinations.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/test.tsv', sep='\t')


In [3]:
def knockout_FBA(gene_ids):
    result = []
    with model:
        for gene_id in gene_ids:
            model.genes.get_by_id(gene_id).knock_out()
        result.append(model.optimize())

    return result

combinations['results'] = combinations['gene_model_ids'].apply(knockout_FBA)
combinations.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/test.tsv', sep='\t')

In [3]:
# Protected class, not sure if that is good, or if it works, quickly eats up memory.
def knockout_FBA_alt(gene_ids):
    return cobra.flux_analysis.deletion._multi_deletion(model, 'gene', cobra.flux_analysis.deletion._element_lists(model.genes, gene_ids), processes=4)


gene_id_list = combinations['gene_model_ids'].to_list()
knockout_FBA_alt(gene_id_list).to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/test2.tsv', sep='\t')
cobra.flux_analysis.

TypeError: unhashable type: 'list'

In [ ]:
def get_model_gene_info():
    ids = model.genes.list_attr('id')
    gene_names = []

    for i in ids:
        gene_names.append(model.genes.get_by_id(i).__getattribute__('name'))

    pd.Series(gene_names).to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/gene_names_in_model.tsv', sep='\t')

In [75]:
#print(model.reactions.HMR_9505.gene_name_reaction_rule)
#model.reactions.MAOX.genes


gene = model.genes.get_by_id('26_AT1')
print(gene.__getattribute__('name'))
gene


AOC1


Gene identifier,26_AT1
Name,AOC1
Memory address,0x013f1ef16940
Functional,True
In 15 reaction(s),"PEAMNO, RE0827E, MHISOR, PTRCOX1, RE0690E, RE3367E, TRYPTAOX, 42A12BOOX, RE0689E, 13DAMPPOX, RE0688E, r0281, RE0828E, MAOX, HISTASE"
